In [10]:
import numpy as np

class FuzzyVariable:
    def __init__(self, name, ranges):
        self.name = name
        self.ranges = ranges

    def fuzzify(self, value):
        memberships = {}
        for term, (a, b, c, d) in self.ranges.items():
            if a <= value <= b:
                memberships[term] = (value - a) / (b - a)
            elif b <= value <= c:
                memberships[term] = 1
            elif c <= value <= d:
                memberships[term] = (d - value) / (d - c)
            else:
                memberships[term] = 0
        return memberships

class FuzzyRule:
    def __init__(self, conditions, output):
        self.conditions = conditions
        self.output = output

    def evaluate(self, memberships):
        condition_values = [memberships[var][term] for var, term in self.conditions.items()]
        return min(condition_values)

class FuzzyInferenceSystem:
    def __init__(self, variables, rules):
        self.variables = variables
        self.rules = rules

    def infer(self, inputs):
        fuzzy_values = {var: self.variables[var].fuzzify(value) for var, value in inputs.items()}
        outputs = {}
        for rule in self.rules:
            rule_strength = rule.evaluate(fuzzy_values)
            output_term, output_range = rule.output
            if output_term not in outputs:
                outputs[output_term] = []
            outputs[output_term].append((rule_strength, output_range))

        aggregated = {}
        for term, values in outputs.items():
            total_strength = sum(strength for strength, _ in values)
            if total_strength > 0:
                aggregated[term] = sum(strength * np.mean(output_range) for strength, output_range in values) / total_strength
            else:
                aggregated[term] = 0
        return aggregated

# Defining fuzzy variables
permintaan = FuzzyVariable("Permintaan", {
    "turun": (0, 0, 2000, 4000),
    "tetap": (2000, 4000, 4000, 6000),
    "naik": (4000, 6000, 8000, 8000)
})

persediaan = FuzzyVariable("Persediaan", {
    "sedikit": (0, 0, 500, 1000),
    "sedang": (500, 1000, 1000, 1500),
    "banyak": (1000, 1500, 2000, 2000)
})

produksi = FuzzyVariable("Produksi", {
    "berkurang": (0, 0, 3000, 7000),
    "bertambah": (3000, 7000, 15000, 15000)
})

# Defining fuzzy rules
rules = [
    FuzzyRule({"Permintaan": "turun", "Persediaan": "sedikit"}, ("bertambah", (10000, 15000))),
    FuzzyRule({"Permintaan": "turun", "Persediaan": "sedang"}, ("bertambah", (7000, 10000))),
    FuzzyRule({"Permintaan": "turun", "Persediaan": "banyak"}, ("berkurang", (3000, 6000))),
    FuzzyRule({"Permintaan": "tetap", "Persediaan": "sedikit"}, ("bertambah", (7000, 12000))),
    FuzzyRule({"Permintaan": "tetap", "Persediaan": "sedang"}, ("berkurang", (2000, 5000))),
    FuzzyRule({"Permintaan": "tetap", "Persediaan": "banyak"}, ("berkurang", (3000, 7000))),
    FuzzyRule({"Permintaan": "naik", "Persediaan": "sedikit"}, ("bertambah", (12000, 15000))),
    FuzzyRule({"Permintaan": "naik", "Persediaan": "sedang"}, ("berkurang", (5000, 7000))),
    FuzzyRule({"Permintaan": "naik", "Persediaan": "banyak"}, ("berkurang", (3000, 7000)))
]

# Creating Fuzzy Inference System
fis = FuzzyInferenceSystem({"Permintaan": permintaan, "Persediaan": persediaan}, rules)

# Example input sets
example_inputs = [
    {"Permintaan": 2500, "Persediaan": 800},
    {"Permintaan": 1500, "Persediaan": 400},
    {"Permintaan": 6000, "Persediaan": 1200},
    {"Permintaan": 4500, "Persediaan": 700},
    {"Permintaan": 7500, "Persediaan": 1000}
]

# Evaluate and print outputs for all examples
for i, inputs in enumerate(example_inputs):
    output = fis.infer(inputs)
    formatted_output = {k: float(v) for k, v in output.items()}  # Convert np.float64 to float
    print(f"Example {i + 1} Inputs: {inputs} => Output: {formatted_output}")


Example 1 Inputs: {'Permintaan': 2500, 'Persediaan': 800} => Output: {'bertambah': 9980.0, 'berkurang': 3500.0}
Example 2 Inputs: {'Permintaan': 1500, 'Persediaan': 400} => Output: {'bertambah': 12500.0, 'berkurang': 0.0}
Example 3 Inputs: {'Permintaan': 6000, 'Persediaan': 1200} => Output: {'bertambah': 0.0, 'berkurang': 5600.0}
Example 4 Inputs: {'Permintaan': 4500, 'Persediaan': 700} => Output: {'bertambah': 10676.470588235294, 'berkurang': 4461.538461538461}
Example 5 Inputs: {'Permintaan': 7500, 'Persediaan': 1000} => Output: {'bertambah': 0.0, 'berkurang': 6000.0}
